In [1]:
import json
import numpy as np
import os
import paramiko
import sklearn
import subprocess

print(np.__version__)
print(sklearn.__version__)

os.chdir('..')

1.18.5
0.24.1


## LinTS GoodReads Recommendations

This notebook explores the differences in GoodReads recommendations across multiple NumPy environments. In particular, this notebook uses the preprocessed data generated in [Goodreads Preprocessing](Goodreads%20Preprocessing.ipynb) with some sampling for time constraints, as in [Goodreads Samples](Goodreads%20Samples.ipynb). All scenarios use LinTS for generating recommendations.

In [2]:
%%writefile example_lints_goodreads.py

# Run this cell for every experiment
from datetime import datetime
import json
import pandas as pd
import numpy as np
import os
import platform
import pickle
from sklearn.preprocessing import StandardScaler
import sys

from mabwiser.mab import MAB
from mabwiser.linear import _RidgeRegression, _Linear

from utils import all_positive_definite

random_option = sys.argv[1]

class LinTSExample(_RidgeRegression):
    def predict(self, x):
        if self.scaler is not None:
            x = self._scale_predict_context(x) 
        if random_option == 'cholesky':
            beta_sampled = rng2.multivariate_normal(self.beta, self.A_inv, method='cholesky')
        else:
            beta_sampled = rng2.multivariate_normal(self.beta, self.A_inv)
        return np.dot(x, beta_sampled)
    
class LinearExample(_Linear):
    factory = {"ts": LinTSExample}

    def __init__(self, rng, arms, n_jobs=1, backend=None, l2_lambda=1, alpha=1, regression='ts', arm_to_scaler = None):
        super().__init__(rng, arms, n_jobs, backend, l2_lambda, alpha, regression)
       
        self.l2_lambda = l2_lambda
        self.alpha = alpha
        self.regression = regression

        # Create ridge regression model for each arm
        self.num_features = None

        if arm_to_scaler is None:
            arm_to_scaler = dict((arm, None) for arm in arms)

        self.arm_to_model = dict((arm, LinearExample.factory.get(regression)(rng, l2_lambda,
                                                                       alpha, arm_to_scaler[arm])) for arm in arms)


# base_path = os.path.dirname(__file__)
base_path = 'goodreads'

# Dataset 1
users = pd.read_csv(os.path.join(base_path, 'sample_user_features.csv.gz'))
responses = pd.read_csv(os.path.join(base_path, 'sample_responses.csv.gz'))
train = users[users['set']=='train']
test = users[users['set']=='test']

train = train.merge(responses, how='left', on='user_id')

context_features = [c for c in users.columns if c not in ['user_id', 'set']]

decisions = MAB._convert_array(train['book_id'])
rewards = MAB._convert_array(train['response'])
contexts = MAB._convert_matrix(train[context_features]).astype('float')
test_contexts = MAB._convert_matrix(test[context_features]).astype('float')

scaler = StandardScaler()
contexts = scaler.fit_transform(contexts)
test_contexts = scaler.transform(test_contexts)
item_ids = list(responses['book_id'].unique())

if random_option == 'randomstate':
    rng = np.random.RandomState(seed=11)
    rng2 = rng
elif random_option == 'svd':
    rng = np.random.RandomState(seed=11)
    rng2 = np.random.default_rng(11)
elif random_option == 'cholesky':
    rng = np.random.RandomState(seed=11)
    rng2 = np.random.default_rng(11)

mab = LinearExample(rng=rng, arms=item_ids, l2_lambda=1, alpha=1, regression='ts', n_jobs=1, backend=None)


np.random.seed(42)
mab.fit(decisions, rewards, contexts)
print(all_positive_definite(mab))
exps = mab.predict_expectations(test_contexts)

recs = [max(user_exps, key=user_exps.get).item() for user_exps in exps]
print(recs)

Overwriting example_lints_goodreads.py


We try training a MAB model and getting a recommendation from the model across 2 different environments:
1. OpenBLAS-backed NumPy on MacOS Catalina
2. MKL-backed NumPy on MacOS Catalina

All environments contain NumPy version 1.18.5.

In [3]:
with open('ssh_config.json') as fp:
    conf = json.load(fp)

local_conf = {'host_name': 'localhost'}

envs = [
    ('mac_openblas', local_conf, os.path.expanduser('~/Tools/miniconda3/envs/reprod/bin/python')),
    ('mac_mkl', local_conf, os.path.expanduser('~/Tools/miniconda3/envs/reprod2/bin/python')),
#     ('linux_openblas', conf, '$HOME/Tools/miniconda3/envs/reprod/bin/python'),
#     ('linux_mkl', conf, '$HOME/Tools/miniconda3/envs/reprod2/bin/python'),
]

def test_envs(env_lis, script, option):
    all_vals = []
    for env_name, conf, python_exec in env_lis:
        print(f'Running {env_name}...')
        if conf['host_name'] == 'localhost':
            res = subprocess.run([python_exec, script, option], capture_output=True).stdout.decode('utf-8').strip()
            all_positive_definite, res = res.split('\n')
            print(f"All covariances positive definite: {all_positive_definite}")
            all_vals.append(eval(res))
        else:
            with paramiko.SSHClient() as ssh:
                ssh.load_system_host_keys()
                ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
                ssh.connect(hostname=conf['host_name'], username=conf['username'], key_filename=os.path.expanduser(conf['key_filename']))
                with ssh.open_sftp() as ftp:
                    ftp.put(script, f'/tmp/{script}')
                    ftp.put('utils.py', '/tmp/utils.py')
                    ftp.put('movielens_users.csv', '/tmp/movielens_users.csv')
                    ftp.put('movielens_responses.csv', '/tmp/movielens_responses.csv')

                _, stdout, stderr = ssh.exec_command(f'{python_exec} /tmp/{script} {option}', get_pty=True)

                # Log the stdout as it comes
                all_positive_definite, res = [line.strip() for line in stdout.readlines()]
                print(f"All covariances positive definite: {all_positive_definite}")
                all_vals.append(eval(res))

    return np.unique(all_vals, axis=0)

### Scenario 1

We test Goodreads data with the legacy `RandomState` class across the 4 different environments.

In [4]:
test_envs(envs, 'example_lints_goodreads.py', 'randomstate')

Running mac_openblas...
All covariances positive definite: True
Running mac_mkl...
All covariances positive definite: True


array([[ 2914097,    99561,    24768, ..., 15749186, 23395680,   119324],
       [ 2914097,  2914097,    24768, ...,   693208, 23395680, 23395680]])

As we can see, the top 1 recommendations generated for each user can be quite different across multiple runs. In fact, these 4 different environments lead to 3 different sets of recommendations, showing clearly that setting the seed isn't enough for reproducibility here.

### Scenario 2
We test Goodreads data using the new `Generator` class with default arguments across 4 different environments.

In [5]:
test_envs(envs, 'example_lints_goodreads.py', 'svd')

Running mac_openblas...
All covariances positive definite: True
Running mac_mkl...
All covariances positive definite: True


array([[  119324,    22232,  9593911, ...,  4502507,   345627,  1326258],
       [13047090, 14061955, 18460392, ...,  7937462,   345627,  6527740]])

We can see that even with the new `Generator` class, which uses SVD decomposition by default, we still have a problem, with 4 different environments giving 3 different set of recommendations.

### Scenario 3
We test Goodreads data using the new `Generator` class with Cholesky decomposition method, across 4 different environments.

In [6]:
test_envs(envs, 'example_lints_goodreads.py', 'cholesky')

Running mac_openblas...
All covariances positive definite: True
Running mac_mkl...
All covariances positive definite: True


array([[   51738,  9593911,  9961796, ...,  9520360, 13414446, 10637766]])

We can see that using Cholesky decomposition, there is only one single set of recommendations the users receive, further showing that Cholesky decomposition is important for reproducibility purposes.